INSTALANDO SELENIUM

In [1]:
#pip install selenium

IMPORTANTO BIBLIOTECAS

In [1]:
#BIBLIOTECAS
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


Url para o web scraping

In [2]:
url = 'https://sagrescidadao.tce.pb.gov.br/#/home-estadual'

Criando o driver

In [3]:
service = Service()

options = webdriver.FirefoxOptions()

driver = webdriver.Firefox(options=options, service=service)
driver.get('https://sagrescidadao.tce.pb.gov.br/#/home-estadual')

Acessando o caminho para o seção da secretaria de saúde

In [4]:
driver.find_elements(By.TAG_NAME, "p-dropdown")[1].click()
driver.find_elements(By.TAG_NAME, "li")[3].click()
driver.find_element(By.TAG_NAME, "p-multiselect").click()
driver.find_element(By.CLASS_NAME, "p-multiselect-filter").send_keys('Secretaria de Estado da Saúde')
driver.find_elements(By.CLASS_NAME, "p-ripple")[0].click()
driver.find_elements(By.CLASS_NAME, "svg-inline--fa")[0].click()
driver.find_elements(By.CLASS_NAME, "header")[0].click()
driver.find_elements(By.TAG_NAME, "button")[2].click()
driver.find_elements(By.TAG_NAME, "a")[12].click()
#clicar no ícone "next" para passar duas páginas
driver.find_elements(By.XPATH, '//span[@class="ag-icon ag-icon-next"]')[0].click()
driver.find_elements(By.XPATH, '//span[@class="ag-icon ag-icon-next"]')[0].click()
#driver.find_elements(By.CLASS_NAME, "ag-button")[2].click() #1ª
#driver.find_elements(By.CLASS_NAME, "ag-button")[2].click() #2ª
time.sleep(1)
driver.find_elements(By.XPATH, '//span[@class="ag-group-value"]')[3].click() #clicar na aba "PBSAUDE"




ESTRUTURA DE REPETIÇÃO PARA CAPTAÇÃO DOS DADOS

O objetivo dessa etapa é realizar uma iteração sobre as 607 páginas abrindo e coletando os dados das 5 tabelas contidas em cada página.

In [5]:
#abrindo a aba de "CLT - ATIVO"
time.sleep(2)
driver.find_elements(By.XPATH, '//span[@class="ag-group-value"]')[11].click()


In [6]:
#Páginas
start_page = int(driver.find_elements(By.XPATH, '//span[@class="ag-paging-number"]')[0].text)
last_page = int(driver.find_elements(By.XPATH, '//span[@class="ag-paging-number"]')[1].text)

'''Início e fim do range para iteração sob as páginas'''
    

'Início e fim do range para iteração sob as páginas'

In [7]:
#lista para armazenar dados coletados
clt_data=[]

#Páginas
start_page = int(driver.find_elements(By.XPATH, '//span[@class="ag-paging-number"]')[0].text)
last_page = int(driver.find_elements(By.XPATH, '//span[@class="ag-paging-number"]')[1].text)

#iteração sobre as páginas
for n in range(start_page,last_page+1):

  #aguardando a página carregar
  wait = WebDriverWait(driver, 20)
  element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn-detalhes')))
  '''
  1 - Problema no loop da iteração de páginas, precisa de um range customizado. RESOLVIDO
  2 - Problema no carregamento de página similar ao carregamento de tabela, é preciso esperar a página carregar. RESOLVIDO
  '''
  
  #ícones para clicar (tabelas)
  tables = driver.find_elements(By.CSS_SELECTOR, 'button.btn-detalhes')
  
  #iteração nas tabelas
  for table in tables:
    
    #clicar em cada ícone
    table.click()
    #tempo para carregar
    time.sleep(1)
    #lista com elementes para salario
    elements = driver.find_elements(By.TAG_NAME, "td")

    while len(elements) == 0: 
      driver.find_element(By.ID, "botao-fechar").click()#fechar janela
      time.sleep(1)
      table.click() #clicar novamente
      time.sleep(1)
      elements = driver.find_elements(By.TAG_NAME, "td")
    else:
      pass
    
    time.sleep(1)
    #dic com dados
    detalhes_clt = { "cargo":driver.find_elements(By.TAG_NAME, "td")[2].text,
                  "Admissão":driver.find_elements(By.TAG_NAME, "td")[3].text,
                  "Poder":driver.find_elements(By.TAG_NAME, "td")[4].text,
                  "Unidade_gestora":driver.find_elements(By.TAG_NAME, "td")[5].text,
                  #salários
                  "meses-salario":[elements[n].text for n in range(6, len(elements))],
                  "Total":driver.find_elements(By.TAG_NAME, "th")[7].text }
    clt_data.append(detalhes_clt) #adicionando cada dicionário à lista clt_data
    driver.find_element(By.ID, "botao-fechar").click()#fechar janela
    time.sleep(2)
  driver.find_element(By.XPATH, '//span[@class="ag-icon ag-icon-next"]').click() #próxima aba
 

CRIANDO DATAFRAME

Essa etapa final do scraping é dedicada a criação do dataframe para posterior análise de dados. 

In [8]:
#Verificando a lista de dicionarios criada
len(clt_data)

677

In [9]:
#Criando o dataframe
df = pd.DataFrame(clt_data)
df

,cargo,Admissão,Poder,Unidade_gestora,meses-salario,Total
0,TECNICO DE ENFERMAGEM EM TERAPIA INTENSI,27/07/2023,Executivo,PBSAUDE,"[Março, R$ 4.133,27, Maio, R$ 4.156,15, Fevere...","R$ 12.381,28"
1,TECNICO EM RADIOLOGIA,01/01/1900,Executivo,PBSAUDE,"[Março, R$ 1.583,68, Maio, R$ 3.582,43]","R$ 5.166,11"
2,FONOAUDIOLOGO,28/08/2023,Executivo,PBSAUDE,"[Março, R$ 2.172,40, Maio, R$ 2.172,40, Fevere...","R$ 6.517,20"
3,TECNICO DE ENFERMAGEM,07/11/2023,Executivo,PBSAUDE,"[Março, R$ 4.101,96, Maio, R$ 4.126,05, Fevere...","R$ 11.923,29"
4,PSICOLOGO,18/05/2022,Executivo,PBSAUDE,"[Fevereiro, R$ 2.732,71, Maio, R$ 2.231,91, Ma...","R$ 7.214,37"
...,...,...,...,...,...,...
672,TECNICO DE ENFERMAGEM,17/03/2022,Executivo,PBSAUDE,"[Fevereiro, R$ 5.197,91, Março, R$ 4.864,48, M...","R$ 15.769,67"
673,TECNICO EM RADIOLOGIA,01/01/1900,Executivo,PBSAUDE,"[Maio, R$ 6.246,18, Março, R$ 1.583,68]","R$ 7.829,86"
674,TECNICO DE ENFERMAGEM,10/06/2022,Executivo,PBSAUDE,"[Fevereiro, R$ 4.673,52, Maio, R$ 4.126,05, Ma...","R$ 13.335,43"
675,TECNICO DE ENFERMAGEM,08/03/2022,Executivo,PBSAUDE,"[Maio, R$ 4.126,05, Março, R$ 4.133,27, Fevere...","R$ 12.351,18"


In [10]:
#Cópia de segurança
copia_df = df.copy()

In [11]:

copia_df.to_csv("tce_clt_dados_607")

DATAFRAME

Nessa etapa pós coleta de dados construo o dataframe a ser utilizado na análise.

In [22]:
df_clt_tce_2 = pd.read_csv('E:/Data Science/Projetos/Web scraping TCE com selenium/tce_clt_dados_472', index_col=0)
df_clt_tce_1 = pd.read_csv('E:/Data Science/Projetos/Web scraping TCE com selenium/tce_clt_dados_607', index_col=0)

In [23]:
df_clt_tce_2

,cargo,Admissão,Poder,Unidade_gestora,meses-salario,Total
0,MEDICO CARDIOLOGISTA INTERVENCIONISTA -,19/07/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 12.712,40', 'Março', 'R$ 11....","R$ 45.331,14"
1,ENFERMEIRO,27/07/2023,Executivo,PBSAUDE,"['Março', 'R$ 5.684,49', 'Maio', 'R$ 6.796,44'...","R$ 18.148,51"
2,AUXILIAR ADMINISTRATIVO,01/01/1900,Executivo,PBSAUDE,"['Fevereiro', 'R$ 6.016,00', 'Março', 'R$ 6.93...","R$ 18.965,44"
3,MAQUEIRO,01/01/1900,Executivo,PBSAUDE,"['Maio', 'R$ 6.301,29', 'Fevereiro', 'R$ 3.467...","R$ 13.469,67"
4,TECNICO EM RADIOLOGIA,12/09/2023,Executivo,PBSAUDE,"['Março', 'R$ 3.927,98', 'Maio', 'R$ 4.320,63'...","R$ 12.429,84"
...,...,...,...,...,...,...
2350,NUTRICIONISTA,04/08/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 3.003,56', 'Março', 'R$ 3.07...","R$ 8.724,10"
2351,ENFERMEIRO,21/07/2023,Executivo,PBSAUDE,"['Fevereiro', 'R$ 5.667,58', 'Março', 'R$ 5.68...","R$ 17.026,48"
2352,ENFERMEIRO,04/05/2023,Executivo,PBSAUDE,"['Fevereiro', 'R$ 5.667,58', 'Maio', 'R$ 5.674...","R$ 17.026,48"
2353,TECNICO DE ENFERMAGEM,18/01/2024,Executivo,PBSAUDE,"['Maio', 'R$ 4.126,05', 'Fevereiro', 'R$ 3.724...","R$ 12.077,58"


In [24]:
df_clt_tce_1

,cargo,Admissão,Poder,Unidade_gestora,meses-salario,Total
0,TECNICO DE ENFERMAGEM EM TERAPIA INTENSI,27/07/2023,Executivo,PBSAUDE,"['Março', 'R$ 4.133,27', 'Maio', 'R$ 4.156,15'...","R$ 12.381,28"
1,TECNICO EM RADIOLOGIA,01/01/1900,Executivo,PBSAUDE,"['Março', 'R$ 1.583,68', 'Maio', 'R$ 3.582,43']","R$ 5.166,11"
2,FONOAUDIOLOGO,28/08/2023,Executivo,PBSAUDE,"['Março', 'R$ 2.172,40', 'Maio', 'R$ 2.172,40'...","R$ 6.517,20"
3,TECNICO DE ENFERMAGEM,07/11/2023,Executivo,PBSAUDE,"['Março', 'R$ 4.101,96', 'Maio', 'R$ 4.126,05'...","R$ 11.923,29"
4,PSICOLOGO,18/05/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 2.732,71', 'Maio', 'R$ 2.231...","R$ 7.214,37"
...,...,...,...,...,...,...
672,TECNICO DE ENFERMAGEM,17/03/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 5.197,91', 'Março', 'R$ 4.86...","R$ 15.769,67"
673,TECNICO EM RADIOLOGIA,01/01/1900,Executivo,PBSAUDE,"['Maio', 'R$ 6.246,18', 'Março', 'R$ 1.583,68']","R$ 7.829,86"
674,TECNICO DE ENFERMAGEM,10/06/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 4.673,52', 'Maio', 'R$ 4.126...","R$ 13.335,43"
675,TECNICO DE ENFERMAGEM,08/03/2022,Executivo,PBSAUDE,"['Maio', 'R$ 4.126,05', 'Março', 'R$ 4.133,27'...","R$ 12.351,18"


In [28]:
#concatenando
df_clt_tce_final = pd.concat([df_clt_tce_2,df_clt_tce_1], ignore_index=True)
df_clt_tce_final

,cargo,Admissão,Poder,Unidade_gestora,meses-salario,Total
0,MEDICO CARDIOLOGISTA INTERVENCIONISTA -,19/07/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 12.712,40', 'Março', 'R$ 11....","R$ 45.331,14"
1,ENFERMEIRO,27/07/2023,Executivo,PBSAUDE,"['Março', 'R$ 5.684,49', 'Maio', 'R$ 6.796,44'...","R$ 18.148,51"
2,AUXILIAR ADMINISTRATIVO,01/01/1900,Executivo,PBSAUDE,"['Fevereiro', 'R$ 6.016,00', 'Março', 'R$ 6.93...","R$ 18.965,44"
3,MAQUEIRO,01/01/1900,Executivo,PBSAUDE,"['Maio', 'R$ 6.301,29', 'Fevereiro', 'R$ 3.467...","R$ 13.469,67"
4,TECNICO EM RADIOLOGIA,12/09/2023,Executivo,PBSAUDE,"['Março', 'R$ 3.927,98', 'Maio', 'R$ 4.320,63'...","R$ 12.429,84"
...,...,...,...,...,...,...
3027,TECNICO DE ENFERMAGEM,17/03/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 5.197,91', 'Março', 'R$ 4.86...","R$ 15.769,67"
3028,TECNICO EM RADIOLOGIA,01/01/1900,Executivo,PBSAUDE,"['Maio', 'R$ 6.246,18', 'Março', 'R$ 1.583,68']","R$ 7.829,86"
3029,TECNICO DE ENFERMAGEM,10/06/2022,Executivo,PBSAUDE,"['Fevereiro', 'R$ 4.673,52', 'Maio', 'R$ 4.126...","R$ 13.335,43"
3030,TECNICO DE ENFERMAGEM,08/03/2022,Executivo,PBSAUDE,"['Maio', 'R$ 4.126,05', 'Março', 'R$ 4.133,27'...","R$ 12.351,18"


In [29]:
df_clt_tce_final.to_csv("clt_tce_dados")